In [4]:
import os
import albumentations
import pandas as pd
import numpy as np

import tez
from tez.datasets import ImageDataset

import torch
import torch.nn as nn
from torch.nn import functional as F

from efficientnet_pytorch import EfficientNet

In [5]:
test_aug = albumentations.Compose([
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)

In [7]:
dfx_test = pd.read_csv("Snakes in Hood/dataset/test.csv")
image_path = "Snakes in Hood/dataset/test/"
test_image_paths = [os.path.join(image_path, x+'.jpg') for x in dfx_test.image_id.values]
# fake targets
dfx_test['breed']=[1]*len(dfx_test)
test_targets = dfx_test.breed.values
test_dataset = ImageDataset(
    image_paths=test_image_paths,
    targets=test_targets,
    resize=(255, 255),
    augmentations=test_aug
)

In [8]:
class SnakeModel(tez.Model):
    def __init__(self, num_classes):
        super().__init__()

        self.effnet = EfficientNet.from_name("efficientnet-b4")
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1792, num_classes)
        self.step_scheduler_after = "epoch"

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs, None, None

In [9]:
model = SnakeModel(num_classes=35)
model.load("Models/model.bin")

AssertionError: Torch not compiled with CUDA enabled